In [3]:
%load_ext autoreload
%autoreload 2

import torch

from text3d2video.utilities.video_util import pil_frames_to_clip
torch.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from text3d2video.pipelines.generative_rendering_pipeline import (
    GrPipeline
)
from text3d2video.pipelines.pipeline_utils import load_pipeline

device = torch.device("cuda")
dtype = torch.float16
sd_repo = "runwayml/stable-diffusion-v1-5"
controlnet_repo = "lllyasviel/control_v11f1p_sd15_depth"
pipe: GrPipeline = load_pipeline(GrPipeline, sd_repo, controlnet_repo)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
# Handstand
from text3d2video.artifacts.anim_artifact import AnimationArtifact

anim = AnimationArtifact.from_wandb_artifact_tag("ymca_zoom_out:latest")
seq = anim.read_anim_seq()

In [13]:
from text3d2video.utilities.video_comparison import display_vid

prompt = "Deadpool"
seed = 0
uvs = seq.render_rgb_uv_maps()
display_vid(pil_frames_to_clip(uvs), title=prompt)

In [32]:
# Call GR
from pathlib import Path

from text3d2video.pipelines.generative_rendering_pipeline import (
    GenerativeRenderingConfig,
)
from text3d2video.utilities.logging import H5Logger


gr_config = GenerativeRenderingConfig(
    do_pre_attn_injection=True,
    do_post_attn_injection=True,
    feature_blend_alpha=0.8,
    attend_to_self_kv=False,
    mean_features_weight=0.5,
    chunk_size=5,
    num_keyframes=4,
    num_inference_steps=15,
    guidance_scale=7.5,
    controlnet_conditioning_scale=1.0,
)

generator = torch.Generator(device="cuda")
generator.manual_seed(seed)

kf_generator = torch.Generator(device="cuda")
kf_generator.manual_seed(0)

logger = H5Logger(Path("data.h5"))
logger.delete_data()
logger.open_write()
logger.enabled = False

out = pipe(
    prompt,
    seq,
    conf=gr_config,
    generator=generator,
    kf_generator=kf_generator,
)

100%|██████████| 15/15 [00:18<00:00,  1.23s/it]


In [ ]:
from text3d2video.utilities.video_comparison import display_vids

vids = {}
vids[" "] = pil_frames_to_clip(uvs)
vids["GR"] = pil_frames_to_clip(out.images)

vids_list = list(vids.values())
titles = list(vids.keys())
display_vids(vids_list, titles=titles)

: 

In [ ]:
from text3d2video.utilities.video_comparison import video_grid

comparison_vids = [
    (pil_frames_to_clip(uvs), "Geometry"),
    (pil_frames_to_clip(video_frames), "Generative Rendering"),
]

labels = [label for _, label in comparison_vids]
clips = [clip for clip, _ in comparison_vids]

comparison_vid = video_grid(
    [clips],
    x_labels=labels,
)
display_vid(comparison_vid)

ImportError: cannot import name 'display_vid' from 'text3d2video.utilities.ipython_utils' (/home/jorge/thesis/text3d2video/utilities/ipython_utils.py)